In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [2]:
PIPELINE_NAME = "personality-pipeline"
SCHEMA_PIPELINE_NAME = "personality-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Pengolahan Data

### Data Ingestion: Pembuatan komponen `ExampleGen`

Merupakan tahap awal dari ML Pipeline, fungsinya adalah untuk mengambil data mentah dan menjadikannya siap diproses dalam pipeline ML.

Karena dataset yang digunakan berformat CSV maka gunakan komponen CsvExampleGen, Pada tahap ini juga dilakukan split data menjadi 80 train dan 20 eval

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=7),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=3)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [7]:
from pprint import PrettyPrinter
pp = PrettyPrinter()

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

tfrecord_train_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_train_filenames, compression_type='GZIP')

for tfrecord in dataset.take(2):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "Drained_after_socializing"
    value {
      bytes_list {
        value: "No"
      }
    }
  }
  feature {
    key: "Friends_circle_size"
    value {
      float_list {
        value: 13.0
      }
    }
  }
  feature {
    key: "Going_outside"
    value {
      float_list {
        value: 6.0
      }
    }
  }
  feature {
    key: "Personality"
    value {
      bytes_list {
        value: "Extrovert"
      }
    }
  }
  feature {
    key: "Post_frequency"
    value {
      float_list {
        value: 5.0
      }
    }
  }
  feature {
    key: "Social_event_attendance"
    value {
      float_list {
        value: 4.0
      }
    }
  }
  feature {
    key: "Stage_fear"
    value {
      bytes_list {
        value: "No"
      }
    }
  }
  feature {
    key: "Time_spent_Alone"
    value {
      float_list {
        value: 4.0
      }
    }
  }
}

features {
  feature {
    key: "Drained_after_socializing"
    value {
      bytes_list {
        value: "Y

### Data Validation: Ada 3 komponen `StatisticGen`,  `ShcemaGen()`, `ExampleValidator()`

Tujuan: memastikan kualitas dan konsistensi data sebelum digunakan dalam proses pelatihan model machine learning

Summary Statistics berfungsi untuk memberi ringkasan statistik, data schema berfungsi untuk menentukan struktur dan tipe data dataset, dan example validator untuk mendeteksi anomali

In [8]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [9]:
interactive_context.show(statistics_gen.outputs["statistics"])

In [10]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [11]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Drained_after_socializing',STRING,required,,'Drained_after_socializing'
'Friends_circle_size',FLOAT,required,,-
'Going_outside',FLOAT,required,,-
'Personality',STRING,required,,'Personality'
'Post_frequency',FLOAT,required,,-
'Social_event_attendance',FLOAT,required,,-
'Stage_fear',STRING,required,,'Stage_fear'
'Time_spent_Alone',FLOAT,required,,-


,Values
Domain,
'Drained_after_socializing',"'No', 'Yes'"
'Personality',"'Extrovert', 'Introvert'"
'Stage_fear',"'No', 'Yes'"


In [12]:
# TAHAP 3, untuk mengidentifikasi anomali, butuh statistik dan schema sebagai input
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [13]:
interactive_context.show(example_validator.outputs['anomalies'])

### Data Preprocessing: buat komponen `Transform`

Tujuan: mengubah data mentah menjadi data yang siap digunakan untuk melatih model

In [14]:
TRANSFORM_MODULE_FILE = "modules/personality_transform.py"

In [15]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

# Label 
LABEL_KEY = "Personality"

# Fitur kategorikal
CATEGORICAL_FEATURE_KEYS = [
    "Stage_fear",
    "Drained_after_socializing"
]

# Fitur numerik 
NUMERIC_FEATURE_KEYS = [
    "Time_spent_Alone",
    "Social_event_attendance",
    "Going_outside",
    "Friends_circle_size",
    "Post_frequency"
]

def transformed_name(key: str) -> str:
    """Menghasilkan nama fitur setelah transformasi."""
    return key + "_xf"

def preprocessing_fn(inputs: dict) -> dict:
    """
    Fungsi preprocessing untuk mentransformasi fitur (tanpa penanganan missing value).

    Args:
        inputs (dict): Dictionary berisi fitur input (tensors).

    Returns:
        dict: Dictionary berisi fitur output hasil transformasi.
    """
    outputs = {}

    # 1. Proses fitur numerik: langsung normalisasi
    for key in NUMERIC_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.scale_to_z_score(inputs[key])

    # 2. Proses fitur kategorikal: langsung encoding
    for key in CATEGORICAL_FEATURE_KEYS:
        encoded = tft.compute_and_apply_vocabulary(inputs[key])
        outputs[transformed_name(key)] = encoded

    # 3. Label binarisasi
    outputs[transformed_name(LABEL_KEY)] = tf.cast(
        tf.equal(inputs[LABEL_KEY], "Extrovert"), tf.int64
    )

    return outputs


Overwriting modules/personality_transform.py


In [16]:
%pip install wheel==0.43.0 setuptools==69.5.1

Note: you may need to restart the kernel to use updated packages.


In [17]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

In [18]:
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\personality-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\3e5c0c3b1bf84b90a8e35831ab8a3adc\assets


INFO:tensorflow:Assets written to: pipelines\personality-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\3e5c0c3b1bf84b90a8e35831ab8a3adc\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Lakukan Tuning

In [19]:
TUNER_MODULE_FILE = "modules/personality_tuner.py"

In [20]:
%%writefile {TUNER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
import kerastuner
import os

from collections import namedtuple
from tensorflow.keras import layers
from kerastuner import HyperParameters
from tfx.components.trainer.fn_args_utils import FnArgs

TunerFnResult = namedtuple('TunerFnResult', ['tuner', 'fit_kwargs'])

# Label target
LABEL_KEY = "Personality"

# Fitur kategorikal
CATEGORICAL_FEATURE_KEYS = [
    "Stage_fear",
    "Drained_after_socializing"
]

# Fitur numerik
NUMERIC_FEATURE_KEYS = [
    "Time_spent_Alone",
    "Social_event_attendance",
    "Going_outside",
    "Friends_circle_size",
    "Post_frequency"
]

def transformed_name(key: str) -> str:
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs=1, batch_size=32):
    if isinstance(file_pattern, list):
        all_files = []
        for pattern in file_pattern:
            all_files.extend(tf.io.gfile.glob(pattern))
        file_pattern = all_files

    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
        shuffle=True,
        shuffle_buffer_size=1000
    )

    return dataset.prefetch(tf.data.AUTOTUNE)

def build_model(hp):
    inputs = {}
    encoded_inputs = []

    # Input kategorikal dengan embedding
    for key in CATEGORICAL_FEATURE_KEYS:
        input_key = transformed_name(key)
        inputs[input_key] = tf.keras.Input(shape=(1,), name=input_key, dtype=tf.int64)
        embedding = layers.Embedding(input_dim=10, output_dim=4)(inputs[input_key])
        flat = layers.Flatten()(embedding)
        encoded_inputs.append(flat)

    # Input numerik
    for key in NUMERIC_FEATURE_KEYS:
        input_key = transformed_name(key)
        inputs[input_key] = tf.keras.Input(shape=(1,), name=input_key, dtype=tf.float32)
        encoded_inputs.append(inputs[input_key])

    # Gabungkan semua input
    x = layers.Concatenate()(encoded_inputs)
    x = layers.Dense(hp.Int("units_1", 32, 128, step=16), activation='relu')(x)
    x = layers.Dense(hp.Int("units_2", 16, 64, step=16), activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [0.001, 0.01, 0.1])),
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=None, batch_size=32)
    eval_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=None, batch_size=32)

    tuner = kerastuner.RandomSearch(
        build_model,
        max_trials=5,
        objective='val_binary_accuracy',
        directory=fn_args.working_dir,
        project_name='personality_tuning',
        max_consecutive_failed_trials=5
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_set,
            "validation_data": eval_set,
            "steps_per_epoch": 50,
            "validation_steps": 25,
            "epochs": 5,
            "verbose": 1
        }
    )


Overwriting modules/personality_tuner.py


In [21]:
# Tambahkan komponen Tuner ke dalam pipeline
from tfx.components import Tuner
from tfx.proto import trainer_pb2


tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500)
)



interactive_context.run(tuner)


Trial 5 Complete [00h 00m 02s]
val_binary_accuracy: 0.9375

Best val_binary_accuracy So Far: 0.9387500286102295
Total elapsed time: 00h 00m 14s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\personality-pipeline\.temp\6\personality_tuning
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
units_1: 96
units_2: 64
learning_rate: 0.01
Score: 0.9387500286102295

Trial 2 summary
Hyperparameters:
units_1: 80
units_2: 64
learning_rate: 0.01
Score: 0.9387500286102295

Trial 0 summary
Hyperparameters:
units_1: 64
units_2: 48
learning_rate: 0.001
Score: 0.9375

Trial 3 summary
Hyperparameters:
units_1: 96
units_2: 64
learning_rate: 0.1
Score: 0.9375

Trial 4 summary
Hyperparameters:
units_1: 64
units_2: 64
learning_rate: 0.1
Score: 0.9375


ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

## Membuat Tahapan Pengembangan 

Pada tahap ini, kita akan membuat dan melatih model menggunakan komponen **Trainer**.

In [22]:
TRAINER_MODULE_FILE = "modules/personality_trainer.py"

In [23]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import FnArgs
import os
from kerastuner import HyperParameters

# Label target
LABEL_KEY = "Personality"

# Daftar fitur kategorikal & numerik
CATEGORICAL_FEATURE_KEYS = [
    "Stage_fear",
    "Drained_after_socializing"
]

NUMERIC_FEATURE_KEYS = [
    "Time_spent_Alone",
    "Social_event_attendance",
    "Going_outside",
    "Friends_circle_size",
    "Post_frequency"
]

def transformed_name(key: str) -> str:
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs=10, batch_size=64):
    feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=feature_spec,
        reader=gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY),
        num_epochs=num_epochs,
        shuffle=True
    )
    return dataset.prefetch(tf.data.AUTOTUNE)

def model_builder(hp=None):
    inputs = {}
    encoded_inputs = []

    # Handle kategorikal
    for key in CATEGORICAL_FEATURE_KEYS:
        feat_key = transformed_name(key)
        inputs[feat_key] = tf.keras.Input(shape=(1,), name=feat_key, dtype=tf.int64)
        embed = tf.keras.layers.Embedding(input_dim=10, output_dim=4)(inputs[feat_key])
        flat = tf.keras.layers.Flatten()(embed)
        encoded_inputs.append(flat)

    # Handle numerik
    for key in NUMERIC_FEATURE_KEYS:
        feat_key = transformed_name(key)
        inputs[feat_key] = tf.keras.Input(shape=(1,), name=feat_key, dtype=tf.float32)
        encoded_inputs.append(inputs[feat_key])

    # Gabungkan semua input
    x = tf.keras.layers.Concatenate()(encoded_inputs)

    # Gunakan hasil tuning jika ada
    if hp:
        units_1 = hp.Int("units_1", 32, 128, step=16)
        units_2 = hp.Int("units_2", 16, 128, step=16)
        learning_rate = hp.Choice("learning_rate", [0.001, 0.01, 0.1])
    else:
        units_1 = 64
        units_2 = 32
        learning_rate = 0.001

    x = tf.keras.layers.Dense(units_1, activation='relu')(x)
    x = tf.keras.layers.Dense(units_2, activation='relu')(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output)

    # Gunakan hasil tuning
    if fn_args.hyperparameters:
        hparams = HyperParameters.from_config(fn_args.hyperparameters)
        model = model_builder(hp=hparams)
    else:
        model = model_builder()

    # Callback untuk TensorBoard
    tensorboard_cb = tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs'))

    # Training
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        steps_per_epoch=100,
        validation_steps=50,
        epochs=5,
        callbacks=[tensorboard_cb]
    )

    # Simpan model untuk serving
    model.save(
        fn_args.serving_model_dir,
        save_format='tf',
        signatures={
            'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
            )
        }
    )


Overwriting modules/personality_trainer.py


In [24]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500),
    hyperparameters=tuner.outputs['best_hyperparameters']  
)

interactive_context.run(trainer)


Epoch 1/5
100/100 [==============================] - 1s 6ms/step - loss: 0.2773 - binary_accuracy: 0.9250 - val_loss: 0.2344 - val_binary_accuracy: 0.9378
Epoch 2/5
100/100 [==============================] - 0s 4ms/step - loss: 0.2505 - binary_accuracy: 0.9295 - val_loss: 0.2244 - val_binary_accuracy: 0.9378
Epoch 3/5
100/100 [==============================] - 0s 4ms/step - loss: 0.2407 - binary_accuracy: 0.9294 - val_loss: 0.2242 - val_binary_accuracy: 0.9378
Epoch 4/5
  1/100 [..............................] - ETA: 0s - loss: 0.1789 - binary_accuracy: 0.9688WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 500 batches). You may need to use the repeat() function when building your dataset.


100/100 [==============================] - 3s 26ms/step - loss: 0.2407 - binary_accuracy: 0.9273 - val_loss: 0.2189 - val_binary_accuracy: 0.9375
INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\personality-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\personality-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Membuat Tahapan Analisis dan Validasi Model 

ada 2 buah komponen: `Resolver()` dan `Evaluator()`

### Komponen Resolver

In [25]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Komponen Evaluator

In [26]:
from tfx.components import Evaluator
import tensorflow_model_analysis as tfma

# Konfigurasi evaluasi dengan threshold
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Personality_xf')],
    slicing_specs=[tfma.SlicingSpec()],  # hanya global, tidak slicing
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='BinaryAccuracy')
            ],
            thresholds={
                'binary_accuracy': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.01})
                )
            }
        )
    ]
)


In [27]:
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    eval_config=eval_config
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [28]:
from IPython.display import display

# Ambil URI hasil evaluasi dari komponen evaluator
eval_result_uri = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result_uri)

# Tampilkan metrik per slicing (misalnya seluruh data)
display(tfma.view.render_slicing_metrics(tfma_result))
try:
    display(tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result))
except Exception as e:
    print("Fairness indicator gagal ditampilkan (kemungkinan karena tidak ada fitur slice yang relevan):", e)


SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## Menambahkan Komponen Pusher

Komponen ini akan menerima inputan berupa trained model, hasil evaluasi dari komponen Evaluator, dan argumen terkait serving file path

In [29]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model=trainer.outputs['model'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.abspath('serving_model_dir/personality_model')
        )
    )
)
interactive_context.run(pusher)



ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))